# Import Libraries

In [2]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException

# Define Countries to be in the list

In [ ]:
paises_filtrados = ['Alemanha', 'Angola', 'Argélia', 'Argentina', 'Arménia', 'Bélgica', 'Bolívia', 'Brasil', 'Bulgária', 'Camarões', 'Canadá', 'Chile', 'China', 'Colômbia', 'Coreia do Sul', 'Costa do Marfim',
                    'Costa Rica', 'Croácia', 'Dinamarca', 'Egito', 'Emirados Árabes Unidos', 'Equador', 'Escócia', 'Espanha', 'Estados Unidos da América', 'França', 'Gana', 'Grécia', 'Holanda', 'Inglaterra', 
                    'Irlanda', 'Irlanda do Norte', 'Islândia', 'Israel', 'Itália', 'Japão', 'Marrocos', 'México', 'Mónaco', 'Nigéria', 'Noruega', 'Paraguai', 'Perú', 'Polónia', 'Portugal', 'Qatar', 'Quénia', 
                    'RD do Congo', 'Reino Unido', 'República Checa', 'República Popular do Congo', 'Rússia', 'Senegal', 'Sérvia', 'Sérvia e Montenegro', 'Sudão', 'Sudão do Sul', 'Suécia', 'Suiça', 'Tailândia', 
                    'Trinidad e Tobago', 'Tunísia', 'Turquia', 'Ucrânia', 'Uruguai', 'Venezuela']

# Get the data

## Get the country link

### Function to get the url

In [ ]:
def get_country_url(list_paises):
    for p in list_paises:
        chrome_path = 'C:/Users/dmakishidias/Documents/Selenium/chromedriver.exe'
        driver = webdriver.Chrome(executable_path=chrome_path)
        # navigate to webpage
        url = 'https://www.transfermarkt.com.br/'
        driver.get(url)
        # Country drop down
        shadow_section = driver.execute_script('''return document.querySelector("tm-quick-select-bar").shadowRoot''')
        # Click on arrow to show countries
        shadow_section.find_element_by_css_selector('div > form:nth-child(2) > div > div').click()
        # Type the country name
        shadow_section.find_element_by_css_selector('div > form:nth-child(2) > div.selector-dropdown > div > input[type=search]').send_keys(p)
        # Press enter
        shadow_section.find_element_by_css_selector('#mylist > li').click()
        # Get URL
        url = shadow_section.find_element_by_css_selector('div > form:nth-child(2) > a').get_attribute('href')
        # Add to the table
        list_to_add = [p, url]
        paises_url.loc[-1] = list_to_add
        paises_url = paises_url.reset_index(drop=True)
        # close
        driver.close() 
        return paises_url

### Save the data in CSV

In [ ]:
paises_url.to_csv('paises.csv')

## Get the league info

### Save the csv with countries links in a pandas dataframe

In [ ]:
pd_paises_url = pd.read_csv('paises.csv')

### Function to get the url

In [ ]:
def get_league_url(pd_paises_url):
    for p in pd_paises_url.values: 
        chrome_path = 'C:/Users/dmakishidias/Documents/Selenium/chromedriver.exe'
        driver = webdriver.Chrome(executable_path=chrome_path)
        # navigate to webpage
        urlp = p[2] 
        driver.get(urlp)
        try:
            # First League
            liga = driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child(2) > td:nth-child(1)').text 
            url = driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr > td:nth-child(2) > a').get_attribute('href')
            list_to_add = [p[0], liga, url]
            liga_url.loc[-1] = list_to_add
            liga_url = liga_url.reset_index(drop=True)
        except:
            pass
        try:
            # Second League
            liga = driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child(4) > td:nth-child(1)').text 
            url = driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child(4) > td:nth-child(1) > table > tbody > tr > td:nth-child(2) > a').get_attribute('href')
            list_to_add = [p[0], liga, url]
            liga_url.loc[-1] = list_to_add
            liga_url = liga_url.reset_index(drop=True)
        except:
            pass
        driver.close() 
    return liga_url

### Save the data in CSV

In [ ]:
liga_url = pd.read_csv('ligas.csv')

## Get the team info

### Save the csv with league links in a pandas dataframe

In [ ]:
pd_league_url = pd.DataFrame(columns=['liga', 'time', 'url'])

### Function to get the url

In [ ]:
def get_team_url(pd_league_url):
    for p in liga_url.values: 
        chrome_path = 'C:/Users/dmakishidias/Documents/Selenium/chromedriver.exe'
        driver = webdriver.Chrome(executable_path=chrome_path)
        # navigate to webpage
        urlp = p[3] 
        driver.get(urlp)
        try:
            r = len(driver.find_element_by_css_selector('#yw1 > table > tbody').text.split('\n')) + 1 
            for t in range(1, r): 
                xp_begin = '#yw1 > table > tbody > tr:nth-child('
                xp_end = ') > td.hauptlink.no-border-links.show-for-small.show-for-pad > a'
                xp_full = xp_begin + str(t) + xp_end
                time = driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child(1) > td.hauptlink.no-border-links.show-for-small.show-for-pad').text 
                url = driver.find_element_by_css_selector(xp_full).get_attribute('href')
                list_to_add = [p[2], time, url]  
                team_table.loc[-1] = list_to_add
                team_table = team_table.reset_index(drop=True)   
        except:
            pass
        driver.close() 
        return team_table

### Save the data in CSV

In [ ]:
team_table = pd.read_csv('teams.csv')

## Get the player info

### Function to get the url

In [ ]:
def get_player_link2(url_team):
    from selenium import webdriver
    chrome_path = 'C:/Users/dmakishidias/Documents/Selenium/chromedriver.exe'
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.get(url_team)
    player_url = []
    try:
            r = len(driver.find_element_by_css_selector('#yw1 > table > tbody').text.split('\n')) + 1 
            for t in range(1, r): 
                xp_begin = '#yw1 > table > tbody > tr:nth-child('
                xp_end = ') > td.posrela > table > tbody > tr:nth-child(1) > td.hauptlink'
                xp_full = xp_begin + str(t) + xp_end
                atleta = driver.find_element_by_css_selector(xp_full).text 

                xp_begin = '#yw1 > table > tbody > tr:nth-child('
                xp_end = ') > td.posrela > table > tbody > tr:nth-child(1) > td.hauptlink > div:nth-child(1) > span > a'
                xp_full = xp_begin + str(t) + xp_end
                url = driver.find_element_by_css_selector(xp_full).get_attribute('href')

                list_to_add = [atleta, url]  
                player_url.append(list_to_add)        
    except:
        pass
    driver.close() 
    return player_url

### Get the info with multiprocessing

In [ ]:
%%time
pool = multiprocess.Pool(processes=12)
result = pool.map(get_player_link2, tqdm(team_table['url'].unique()))
pool.terminate()

### Transform the list in a pandas dataframe

In [ ]:
for r in result:
    try:
        player_url = pd.concat([tabela_jogadores, r], ignore_index = True)
    except:
        pass

### Get the player info

In [ ]:
def get_player_info(url, df_verificacao):
    if url in df_verificacao['url'].values:
        print('test')
        return [None, None]
    else:
        # ADD PLAYERS FOR LOOP
        import pandas as pd
        from selenium import webdriver
        import time
        from selenium.webdriver.common.by import By
        from selenium.webdriver.support import expected_conditions as EC
        from selenium.webdriver.support.wait import WebDriverWait
        from selenium.common.exceptions import TimeoutException
        tabela_jogadores = pd.DataFrame(columns=['atleta', 'data_nascimento', 'local_nascimento', 'idade', 'altura', 'nacionalidade', 'posição', 'pé', 'empresários', 'clube_atual', 'tempo_clube', 'contrato',
                                                 'ultima_renovacao', 'fornecedor', 'posicao_secundaria', 'valor_atual', 'ultima_alteracao', 'url'])

        chrome_path = 'C:/Users/dmakishidias/Documents/Selenium/chromedriver.exe'
        driver = webdriver.Chrome(executable_path=chrome_path)
        # navigate to webpage
        try:
            driver.get(url)
            player_info = driver.find_element_by_xpath('//*[@id="main"]/div[11]/div[1]/div[2]/div[2]/div[2]/div[2]/table').text
            player_info = player_info.split('\n')
            player_info_dict = {}
            for info in range(len(player_info)):
                player_info[info]
                try:
                    info_list = player_info[info].split(':')
                    player_info_dict[info_list[0]] = info_list[1] 
                except:
                    pass
            # Add player info with try
            try:
                tabela_jogadores.loc[-1, 'atleta'] = driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[1]/div/div[1]/h1').text 
            except:
                try:
                    tabela_jogadores.loc[-1, 'atleta'] = driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[2]/div/div[1]/h1').text 
                except:
                    pass
            try:
                tabela_jogadores.loc[-1, 'data_nascimento'] = player_info_dict['Data de nascimento'].strip()
            except:
                pass   
            try:
                tabela_jogadores.loc[-1, 'local_nascimento'] = player_info_dict['Local de nascimento'].strip()
            except:
                pass  
            try:
                tabela_jogadores.loc[-1, 'idade'] = player_info_dict['Idade'].strip()
            except:
                pass 
            try:
                tabela_jogadores.loc[-1, 'altura'] = player_info_dict['Altura'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'nacionalidade'] = player_info_dict['Nacionalidade'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'posição'] = player_info_dict['Posição'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'pé'] = player_info_dict['Pé'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'empresários'] = player_info_dict['Empresários'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'clube_atual'] = player_info_dict['Clube atual'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'tempo_clube'] = player_info_dict['No time desde'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'contrato'] = player_info_dict['Contrato até'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'ultima_renovacao'] = player_info_dict['Última renovação de contrato'].strip()
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'fornecedor'] = player_info_dict['Fornecedor'].strip()
            except:
                pass  
            try:
                tabela_jogadores.loc[-1, 'valor_atual'] = driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[5]/a').text.split('\n')[0]
            except:
                pass 
            try:
                tabela_jogadores.loc[-1, 'ultima_alteracao'] = driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[5]/a').text.split('\n')[1].split(':')[1] 
            except:
                pass
            try:
                tabela_jogadores.loc[-1, 'posicao_secundaria'] = driver.find_element_by_xpath('//*[@id="main"]/div[11]/div[1]/div[2]/div[2]/div[1]/div/div[2]/div/div[1]/div/div[2]').text.split('\n')[1] 
            except:
                pass
            tabela_jogadores.loc[-1, 'url'] = url
            tabela_jogadores = tabela_jogadores.reset_index(drop=True)

            def get_transferencias(driver):
                import pandas as pd
                from selenium import webdriver
                import time
                from selenium.webdriver.common.by import By
                from selenium.webdriver.support import expected_conditions as EC
                from selenium.webdriver.support.wait import WebDriverWait
                from selenium.common.exceptions import TimeoutException
                # Get the transfers of the player
                try:
                    tabela_transferencias = pd.DataFrame(columns=['atleta', 'temporada', 'data', 'origem', 'destino', 'valor_mercado', 'valor_pago'])
                    if len(driver.find_elements_by_xpath('//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr')) == 0:
                        r = int(len(driver.find_elements_by_xpath('//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr'))) + 1
                    else:
                        r = int(len(driver.find_elements_by_xpath('//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr'))) + 1

                    try:
                        for tr in range(1, r):                                                     
                            list_to_add = []
                            try:
                                list_to_add.insert(0, driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[1]/div/div[1]/h1').text) 
                            except:
                                try:
                                    list_to_add.insert(0, driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[2]/div/div[1]/h1').text) 
                                except:
                                    pass
                            # First item
                            item_table = 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            # Second item
                            item_table = 2
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)  
                            # Third item
                            try:
                                item_table = 5
                                begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                                middle_path = ']/td['
                                end_path = ']'
                                value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                                list_to_add.append(value)  
                            except:
                                item_table = 4
                                begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                                middle_path = ']/td['
                                end_path = ']'
                                value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                                list_to_add.append(value) 
                            # Fourth item
                            item_table += 2
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)  
                            # Fifth item
                            item_table += 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            # Last item
                            item_table += 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[3]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            tabela_transferencias.loc[-1] = list_to_add     
                            tabela_transferencias = tabela_transferencias.reset_index(drop=True)
                    except:
                        for tr in range(1, r):                                                     
                            list_to_add = []
                            try:
                                list_to_add.insert(0, driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[1]/div/div[1]/h1').text) 
                            except:
                                try:
                                    list_to_add.insert(0, driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[2]/div/div[1]/h1').text) 
                                except:
                                    pass
                            # First item
                            item_table = 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            # Second item
                            item_table = 2
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)  
                            # Third item
                            item_table = 4
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            if len(value) > 0:
                                list_to_add.append(value)  
                            else:
                                item_table = 5
                                begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                                middle_path = ']/td['
                                end_path = ']'
                                value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                                list_to_add.append(value) 
                            # Fourth item
                            item_table += 3
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)  
                            # Fifth item
                            item_table += 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            # Last item
                            item_table += 1
                            begin_path = '//*[@id="main"]/div[11]/div[1]/div[4]/div[2]/table/tbody/tr['
                            middle_path = ']/td['
                            end_path = ']'
                            value = driver.find_element_by_xpath(begin_path + str(tr) + middle_path + str(item_table) + end_path).text
                            list_to_add.append(value)
                            tabela_transferencias.loc[-1] = list_to_add     
                            tabela_transferencias = tabela_transferencias.reset_index(drop=True)

                    return tabela_transferencias
                except:
                    return None
            df_transfer = get_transferencias(driver)

            driver.close()
            return (tabela_jogadores, df_transfer)
        except:
            driver.close()
            pass

### Get the info with multiprocessing

In [ ]:
%%time
pool = multiprocess.Pool(processes=12)
result = pool.map(partial(get_player_info, df_verificacao=tabela_jogadores), tqdm(list(player_table['url'])[6000:7000]))
pool.terminate()

### Transform the list in a pandas dataframe

In [ ]:
for r in result:
    try:
        tabela_jogadores = pd.concat([tabela_jogadores, r[0]], ignore_index = True)
        player_transfer = pd.concat([player_transfer, r[1]], ignore_index = True)
    except:
        pass

### Save the data in CSV

In [ ]:
player_transfer.to_csv('transfer0_to_1000.csv', index=False)

In [ ]:
tabela_jogadores.to_csv('tabela0_to_1000.csv', index=False)

## Clean the data

### Load the data to pandas dataframe

In [23]:
player_info_table_v1 = pd.read_csv('tabela0_to_1000.csv')
player_info_table_v2 = pd.read_csv('tabela0_to_2000.csv')
player_info_table_v3 = pd.read_csv('tabela0_to_3000.csv')
player_info_table_v4 = pd.read_csv('tabela0_to_4000.csv')
player_info_table_v5 = pd.read_csv('tabela0_to_5000.csv')
player_info_table_v6 = pd.read_csv('tabela0_to_6000.csv')
player_info_table_v7 = pd.read_csv('tabela0_to_7000.csv')

### Concatenate all the tables

In [24]:
main_playerinfo_table = pd.concat([player_info_table_v7, player_info_table_v6, player_info_table_v5, player_info_table_v4, player_info_table_v3, player_info_table_v2,
                                   player_info_table_v1])

### Drop the duplicates

In [25]:
main_playerinfo_table = main_playerinfo_table.drop_duplicates(subset=['atleta', 'data_nascimento', 'local_nascimento', 'altura', 'empresários'])

### Clean the data

In [33]:
main_playerinfo_table.head(5)

,atleta,data_nascimento,local_nascimento,idade,altura,nacionalidade,posição,pé,empresários,clube_atual,tempo_clube,contrato,ultima_renovacao,fornecedor,posicao_secundaria,valor_atual,ultima_alteracao,url,Unnamed: 0
0,João Veras,26/10/2000,Gama,20.0,NaN,Brasil,Atacante - Centroavante,NaN,Casa Soccer/EL,AA Ponte Preta,10/07/2020,31/12/2022,16/07/2020,NaN,NaN,NaN,NaN,https://www.transfermarkt.com.br/joao-veras/pr...,NaN
1,João Gabriel,22/03/1992,Aracaju,28.0,1.90,Brasil,Goleiro,direito,BBM Sports,EC Vitória,03/01/2019,31/12/2021,NaN,NaN,NaN,NaN,NaN,https://www.transfermarkt.com.br/joao-gabriel/...,NaN
2,Ronaldo,22/08/1996,Salvador,24.0,1.94,Brasil,Goleiro,direito,Antoniu's Assessoria,EC Vitória,10/01/2018,-,11/11/2017,NaN,NaN,50000.0,23/05/2018,https://www.transfermarkt.com.br/ronaldo/profi...,NaN
3,Lucas,08/08/1998,NaN,22.0,NaN,Brasil,Goleiro,NaN,Roni7,EC Vitória,-,-,NaN,NaN,NaN,NaN,NaN,https://www.transfermarkt.com.br/lucas/profil/...,NaN
4,Maurício Ramos,10/04/1985,Piracicaba,35.0,1.82,Brasil,Defensor - Zagueiro,direito,"PW SOCCER CONSULTANCY, S.L",EC Vitória,01/01/2020,31/12/2021,NaN,NaN,Lateral Dir.,300000.0,08/04/2020,https://www.transfermarkt.com.br/mauricio-ramo...,NaN


In [27]:
main_playerinfo_table['valor_atual'] = main_playerinfo_table['valor_atual'].str.replace(' €', '').str.replace(',', '').str.replace('mil', '000').\
                                                            str.replace('mi', '0000').str.replace(' ', '').str.replace('.', '').astype(float)

In [47]:
main_playerinfo_table['valor_atual'] = main_playerinfo_table['valor_atual'].replace(' .', ',')

In [30]:
main_playerinfo_table['altura'] = main_playerinfo_table['altura'].str.replace(',', '.').str.replace(' m', '').astype(float)

In [48]:
main_playerinfo_table['altura'] = main_playerinfo_table['altura'].replace('.', ',')

In [49]:
main_playerinfo_table['valor_atual'] = main_playerinfo_table['valor_atual'].replace('.', ',')

In [34]:
main_playerinfo_table = main_playerinfo_table.drop(columns=['Unnamed: 0'])

### Export the main table to CSV

In [50]:
main_playerinfo_table.to_csv('Player Info.csv')

### Cleaning Transfer Data

In [36]:
transfer_info_table_v1 = pd.read_csv('transfer0_to_1000.csv')
transfer_info_table_v2 = pd.read_csv('transfer0_to_2000.csv')
transfer_info_table_v3 = pd.read_csv('transfer0_to_3000.csv')
transfer_info_table_v4 = pd.read_csv('transfer0_to_4000.csv')
transfer_info_table_v5 = pd.read_csv('transfer0_to_5000.csv')
transfer_info_table_v6 = pd.read_csv('transfer0_to_6000.csv')
transfer_info_table_v7 = pd.read_csv('transfer0_to_7000.csv')

In [37]:
main_transferinfo_table = pd.concat([transfer_info_table_v7, transfer_info_table_v6, transfer_info_table_v5, transfer_info_table_v4, transfer_info_table_v3, transfer_info_table_v2,
                                   transfer_info_table_v1])

In [41]:
main_transferinfo_table = main_transferinfo_table.drop(columns=['Unnamed: 0'])

In [42]:
main_transferinfo_table = main_transferinfo_table.drop_duplicates() 

In [44]:
main_transferinfo_table['valor_pago'] = main_transferinfo_table['valor_pago'].str.replace(' €', '').str.replace(',', '').str.replace('mil', '000').\
                                                            str.replace('mi', '0000').str.replace(' ', '').str.replace('.', '').astype(float)

ValueError: could not convert string to float: '-'

In [45]:
main_transferinfo_table['valor_pago'].value_counts().index 

Index(['-', 'custo zero', '50 mil €', '100 mil €', '200 mil €', '300 mil €',
       '150 mil €', '250 mil €', '500 mil €', '400 mil €',
       ...
       '880 mil €', '5,25 mi. €', '1,15 mi. €', '180 mil €',
       'Valor de empréstimo:\n6,80 mi. €', '9,06 mi. €', '50,00 mi. €',
       'Valor de empréstimo:\n350 mil €', 'Valor de empréstimo:\n40 mil €',
       '3,56 mi. €'],
      dtype='object', length=204)